# Using KNearest Neighbour and brute force algorithm, for a given article, we should be able to recommend top 10 articles that are closest to that article. 

In [7]:
import json 
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize #package for flattening json in pandas df
from sklearn.feature_extraction.text import CountVectorizer


In [5]:
%%time

#df = pd.read_json(r'/opt/models/codefordemocracy/data/immi_articles.json', orient='columns')
df = pd.read_json(r'/opt/trainml/input/immigration_articles.json/immigration_articles.json', orient='columns')
df_clean= df.drop(['_id','_score', '_type'], axis=1)
df_src=df.drop(['_index','_id','_score', '_type'], axis=1)
df_src_norm = json_normalize(df_src['_source'])
df_src_norm_limited=df_src_norm.filter(items=['url', 'site_name','extracted.title', 'extracted.date', 'extracted.text', 'metadata.keywords', 'metadata.description'])


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


CPU times: user 45.8 s, sys: 1.63 s, total: 47.4 s
Wall time: 47.5 s


In [8]:
%%time


extracted_text=df_src_norm_limited['extracted.text']
extracted_text.dropna()
#include only those words that appear in less than max_df = 80% of the documents


count_vect = CountVectorizer(max_df=0.7, min_df=10, stop_words='english')
#count_vect = CountVectorizer(max_df=0.8, min_df=2, stop_words=my_stop_words)

#doc_term_matrix = count_vect.fit_transform(df_src_norm_limited['extracted.text'].values.astype('U'))
doc_term_matrix = count_vect.fit_transform(extracted_text.values.astype('U'))

doc_term_matrix

CPU times: user 36.2 s, sys: 6.7 s, total: 42.9 s
Wall time: 42.9 s


<50004x31897 sparse matrix of type '<class 'numpy.int64'>'
	with 9346374 stored elements in Compressed Sparse Row format>

In [9]:
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(metric='euclidean', algorithm='brute')
model.fit(doc_term_matrix)

NearestNeighbors(algorithm='brute', metric='euclidean')

## Here are passing one article - doc_term_matrix[i] and getting back top n neighbors. 

In [10]:
distances, indices = model.kneighbors(doc_term_matrix[0], n_neighbors=10) # 1st arg: word count vector

In [13]:
#print (doc_term_matrix.join(neighbors, on='id').sort('distance')[['id','name','distance']])

In [15]:
print (indices)

[[    0   508 22632 22943 42559 17544 19351 45024 49568 36827]]


In [18]:
print(extracted_text[22632])

Care workers and nurses from overseas should be exempt from salary caps in the Government's new points-based immigration system after Brexit, say two thirds of the public.

An ICMP poll for think tank British Future found some 63 per cent of people said that there should be exceptions to a salary threshold for people moving to the UK to do important jobs that need doing, such as nurses and care workers.

That sentiment was shared across political divides, with agreement from 62 per cent of 2019 Conservative voters and 67 per cent of Labour voters; 58 per cent of Leave voters and 73 per cent of Remain voters.

It is the key area where the public divides from ministers. More than three-quarters of the public would be happy for the numbers of high-skilled workers coming to the UK from the EU (79 per cent) or outside the EU (77 per cent) to stay the same or increase.

Some 64 per cent would like to see the numbers of international students coming to the UK either remain the same or increas